# Intro to Pandas
http://pandas.pydata.org/

In [ ]:
from IPython.display import Image
Image(url='panda1.jpg')

# Contents
1. What is Pandas
2. Why Pandas
3. Datatypes
4. Getting data into Pandas
5. Merging dataframes
6. Getting subsets of your data (slicing, etc)
7. Plotting

# What is Pandas

"...fast, easy-to-use data wrangling and statistical computing tool..."

I like to think of it as a *dict like object* that can queried in additional SQL-y kind of way. 
A bit like a Django Model instance.

Created by Wes McKinney in 2007.

Built on top of NumPy

DataFrame heavily influenced by R DataFrame

# Why Pandas

Because R ![Scream](scream.png "R")

# Datatypes

1.	**Series**:	    1D labeled homogeneously-typed array [1,2,3,4,5] 
2.	**DataFrame**:	General 2D labeled, size-mutable tabular structure with potentially heterogeneously-typed columns
3.	**Panel**:	    General 3D labeled, also size-mutable array **(not going to be covered in this talk)**

*'...The best way to think about the pandas data structures is as flexible containers for lower dimensional data. For example, DataFrame is a container for Series, and Panel is a container for DataFrame objects. We would like to be able to insert and remove objects from these containers in a dictionary-like fashion.'*


# Series

In [ ]:
import pandas as pd

In [ ]:
# a scalar value
pd.Series(1)

In [ ]:
a = pd.Series([1,2,3,6,7,9])
print(a)
# Accessing elements
# Index look up, Element 0th, 1st element
print(a[0])
# Using a mask !We'll be coming back to this, it's a biggy
a[a > 6]

In [ ]:
a = pd.Series(range(4), index=('a', 'b', 'c', 'd'))
print(a)
a['c']

In [ ]:
data = {'a' : 0., 'b' : 1., 'c' : 2.}
a = pd.Series(data)
print(a)
print(a.b)
print(a['c'])
print(a[1:])

In [ ]:
#Why no error? I thought series where 'homogeneous'
pd.Series(['1', 3, 'c'])

They are :-) Look at the dtype, it's all the same type, an object type

Pandas is built on top of numpy, we should look into that...

In [ ]:
import numpy as np
a = pd.Series(np.random.randn(5))
print(a)

In [ ]:
a.sum()

In [ ]:
a.median()

In [ ]:
a.count()

Series acts very similarly to a ndarray, and is a valid argument to most NumPy functions

In [ ]:
a.append(3) # Error

Told you they were homogeneous

In [ ]:
a = a.append(pd.Series([99,100,22], index=('what', 'the', 'magic')))
a

In [ ]:
a.magic

In [ ]:
a.index

# Getting data into Pandas

* **read_csv**
* read_excel
* read_hdf
* read_sql
* read_json
* read_html
* read_stata
* read_sas
* read_clipboard
* read_pickle

# DataFrame

In [ ]:
users = [(1, "Jean-Luc", "Picard", "Enterprise", "locutus.2366@enterprise.subspace"), 
         (2, "Geordi", "La Forge", "Enterprise", "reading.rainbow@enterprise.subspace"), 
         (3, "Kathryn", "Janeway", "Voyager", "cap.delta.q@voyager.subspace"), 
         (4, "B'Elanna", "Torres", "Voyager", "warp.drives.rule@voyager.subspace"),
         (5, "Benjamin", "Sisko", "DS9", "shut.up.quark@ds9.subspace"),
         (6, "Kira", "Nerys", "DS9", "cardassian.scum@ds9.subspace")
         ]

users = pd.DataFrame(users, columns=['id', 'first_name', 'last_name', 'ship', 'email'])
users

In [ ]:
users.set_index('id')

In [ ]:
import csv
from uuid import uuid4
from datetime import datetime, timedelta
from random import randrange, choice, randint

def random_date(start, end):
    """
    http://stackoverflow.com/a/553448
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

_end_date = datetime.now()
_start_date = _end_date - timedelta(weeks=300)

available_numbers = ['55500001', '55500002', '55500003', '55500004', '55500005', 
                     '55500006', '55500007']


with open('cdrs.csv', 'w', newline='') as csvfile:
    cdrwriter = csv.writer(csvfile, delimiter=',')
    cdrwriter.writerow('id,uuid,caller_number,destination_number,context,start_stamp,end_stamp,duration,billsec,hangup_cause,accountcode,read_codec,user_id'.split(','))

    for _cdr_count in range(1, 20001):
        _cdr_start_dtstamp = random_date(_start_date, _end_date)
        _call_duration = randint(0, 1000)
        billsec = _call_duration - 5 if (_call_duration - 5) >= 0 else 0

        if billsec == 0: 
            hangup_cause = 'No Anwser'
        else: 
            hangup_cause = 'Anwser'

        _cdr_end_dtstamp = _cdr_start_dtstamp + timedelta(seconds=_call_duration)

        user_id = randint(1,6)
        cdrwriter.writerow([_cdr_count, uuid4(), choice(available_numbers), choice(available_numbers), 'FUTILE', _cdr_start_dtstamp, _cdr_end_dtstamp, _call_duration, billsec, hangup_cause, 'Blah', 'Whoop', user_id])


In [ ]:
import pandas as pd
_cdr = pd.read_csv('cdrs.csv')
_cdr.head(3)

In [ ]:
pd.set_option('display.max_columns', 1)
_cdr.head(2)

In [ ]:
# default is 20, back to default 
pd.set_option('display.max_columns', 20)
_cdr.head(1)
users

In [ ]:
_cdr = pd.read_csv('cdrs.csv', usecols=['caller_number', 'destination_number', 'start_stamp' ,'end_stamp', 'duration', 
                                        'billsec', 'hangup_cause', 'user_id'], 
                               dtype={'caller_number': 'int', 'destination_number': 'str'}, 
                               index_col='start_stamp', 
                               parse_dates=['start_stamp', 'end_stamp'])

_cdr.head(3)

In [ ]:
_cdr.info()

# Merging dataframes

1. Merge
 * left	
 * right
 * outer
 * inner
2. Concatenate 
3. Join

http://pandas.pydata.org/pandas-docs/stable/merging.html

In [ ]:
_cdr = _cdr.merge(users, how='inner', left_on='user_id', right_on='id')
_cdr.head(5)

# Getting subsets of your data (slicing, etc)

In [ ]:
type(_cdr['user_id'])

In [ ]:
type(_cdr['user_id'][0])

In [ ]:
_cdr['user_id'][0]

In [ ]:
_cdr[:12]['billsec']

In [ ]:
billsec = _cdr['billsec']
print(type(billsec))
billsec.mean()

In [ ]:
billsec.mean() == _cdr['billsec'].mean()

In [ ]:
_cdr['caller_number'].value_counts()

In [ ]:
_cdr['hangup_cause'].unique()

In [ ]:
_cdr.groupby('hangup_cause').count()

In [ ]:
_cdr.groupby('hangup_cause').user_id.count()

In [ ]:
_cdr['email'] == 'shut.up.quark@ds9.subspace' 

In [ ]:
ben_cdrs = _cdr[_cdr['email'] == 'shut.up.quark@ds9.subspace']
ben_cdrs.head(2)

In [ ]:
ben_cdrs.caller_number.count()

In [ ]:
_cdr[(_cdr['email'] == 'shut.up.quark@ds9.subspace') & (_cdr['hangup_cause'] == 'Anwser')].caller_number.count()

In [ ]:
_cdr[(_cdr['email'] == 'shut.up.quark@ds9.subspace') & (_cdr['hangup_cause'] == 'No Anwser')].caller_number.count()

In [ ]:
_cdr[(_cdr['email'] != 'shut.up.quark@ds9.subspace') & (_cdr['user_id'] != 6 )].caller_number.count()

In [ ]:
_cdr.groupby('caller_number').count().sort_values('destination_number', ascending=False).head(3)

# Plotting


In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)

In [ ]:
billsec = _cdr[:30]['billsec']
billsec.plot(kind='bar')

In [ ]:
_cdr[_cdr['email'] == 'reading.rainbow@enterprise.subspace'][:10].billsec.plot(kind='barh')

In [ ]:
_cdr['user_id'].value_counts().plot()

In [ ]:
_cdr['user_id'].value_counts().plot(kind='pie')

In [ ]:
_cdr.count()